In [2]:
import numpy as np
import pandas as pd
import requests
import json
import time

In [3]:
#url = "https://api.ies.ed.gov/eric/?search=mental%20health%20risks%20in%20construction%20industry&rows=200&format=json&start=0"
#responseJson = requests.get(url).json()

In [4]:
#responseJson

In [5]:
def getEricRecords(search, fields = None, start=0, rows=2000):
    url = 'https://api.ies.ed.gov/eric/?'
    url = url + 'search=' + search + '&rows=' + str(rows) + '&format=json&start=' + str(start)
    if(fields):
        url = url + '&fields=' + ', '.join(fields)
    print(url)
    responseJson = requests.get(url).json()
    #df = pd.DataFrame(responseJson)
    return pd.DataFrame(responseJson)

In [6]:
def getRecordCount(search):
    dataFrame = getEricRecords(search)
    totalRecords = dataFrame.loc['numFound'][0]
    print('Search', search, 'returned', format(totalRecords), 'records')
    return totalRecords

In [7]:
def cleanElementsUsingList(x):
    if(not isinstance(x, list)):
        return x
    if(not x or (len(x) == 1 and x[0] == '')):
        return None
    return ', '.join(x)

In [8]:
def getAllEricRecords(search, fields = None, cleanElements = True):
    startTime = time.time()
    nextFirstRecord = 0
    numRecordsReturnedEachApiCall = 2000
    #totalRecords = getRecordCount(search)
    totalRecords = 10000
    if(totalRecords == 0):
        print ('Search', search, 'has no results')
        return []
        
    while(nextFirstRecord < totalRecords):
        dataFrame = getEricRecords(search, fields, nextFirstRecord)
        if(nextFirstRecord == 0):
            records = pd.DataFrame(dataFrame.loc['docs'][0])
        else:
            records = pd.concat([records, pd.DataFrame(dataFrame.loc['docs'][0])], sort=False, ignore_index=True)
        nextFirstRecord += numRecordsReturnedEachApiCall
    #print('took', '{:,.lf}'.format(time.time() - startTime), 'seconds')
    return records.applymap(cleanElementsUsingList) if cleanElements else records

In [9]:
#search = "mental%20health%20risks%20in%20construction%20industry"
search = 'subject:mental%20health%20risks%20in%20construction%20industry AND publicationdateyear:2021'
records = getAllEricRecords(search)
records.info()

https://api.ies.ed.gov/eric/?search=subject:mental%20health%20risks%20in%20construction%20industry AND publicationdateyear:2021&rows=2000&format=json&start=0
https://api.ies.ed.gov/eric/?search=subject:mental%20health%20risks%20in%20construction%20industry AND publicationdateyear:2021&rows=2000&format=json&start=2000
https://api.ies.ed.gov/eric/?search=subject:mental%20health%20risks%20in%20construction%20industry AND publicationdateyear:2021&rows=2000&format=json&start=4000
https://api.ies.ed.gov/eric/?search=subject:mental%20health%20risks%20in%20construction%20industry AND publicationdateyear:2021&rows=2000&format=json&start=6000
https://api.ies.ed.gov/eric/?search=subject:mental%20health%20risks%20in%20construction%20industry AND publicationdateyear:2021&rows=2000&format=json&start=8000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 666 entries, 0 to 665
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  --

In [10]:
#records.to_csv("/home/shiva/Downloads/eric_response.csv")

In [11]:
records

,id,title,author,description,subject,publicationtype,publicationdateyear,language,issn,publisher,peerreviewed,isbn
0,EJ1337645,Assessment of Current and Future Critical Skil...,"Musonda, Innocent, Okoro, Chioma",Purpose: The role of skills development in att...,"Foreign Countries, Construction Industry, Job ...","Journal Articles, Reports - Research",2021,English,ISSN-2042-3896,"Emerald Publishing Limited. Howard House, Wago...",T,NaN
1,EJ1322994,Establishment of Virtual-Reality-Based Safety ...,"Seo, Hyun Jeong, Park, Gyu Mi, Son, Minjie, Ho...",The current safety education and training syst...,"Safety Education, Computer Simulation, Technol...","Journal Articles, Reports - Research",2021,NaN,EISSN-2227-7102,"MDPI AG. Klybeckstrasse 64, 4057 Basel, Switze...",T,NaN
2,EJ1299322,Investigating Communicative Barriers on Constr...,"Ne'Matullah, Khairul Firdaus, Pek, Lim Seong, ...",The evolution of technology has changed the wa...,"Communication Problems, Barriers, Productivity...","Journal Articles, Reports - Research",2021,English,ISSN-2252-8822,Institute of Advanced Engineering and Science....,T,NaN
3,EJ1311907,Why Australian Female High School Students Do ...,"Carnemolla, Phillippa, Galea, Natalie",Background: The construction industry remains ...,"Foreign Countries, Females, High School Studen...","Journal Articles, Reports - Research",2021,English,ISSN-1069-4730,"Wiley. Available from: John Wiley & Sons, Inc....",T,NaN
4,EJ1284715,Strategies for Improving Quantity Surveyors&ap...,"Kibwami, Nathan, Wesonga, Racheal, Manga, Musa...",Education and training of Quantity Surveyors (...,"Internship Programs, Education Work Relationsh...","Journal Articles, Reports - Research",2021,English,ISSN-1913-9020,Canadian Center of Science and Education. 1595...,T,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
661,ED616785,Designing and Implementing E-School Systems: A...,"Grepon, Benzar Glen S., Baran, Niño T., Gumona...",Purpose: Colleges and Universities have been e...,"Program Design, Program Implementation, Colleg...",Reports - Research,2021,English,ISSN-,NaN,F,NaN
662,EJ1335651,Examining Critical Success Factors Augmenting ...,"Pandita, Alka, Kiran, Ravi",Purpose: Our findings show that the academic c...,"Foreign Countries, Educational Quality, Succes...","Journal Articles, Reports - Research",2021,English,ISSN-2050-7003,"Emerald Publishing Limited. Howard House, Wago...",T,NaN
663,EJ1311844,&quot;COVID-19 and Students Perception about M...,"Anand Shankar Raja, Manivannan, Kallarakal, To...",Purpose: The purpose of this paper is to under...,"Online Courses, COVID-19, Pandemics, Higher Ed...","Journal Articles, Reports - Research, Tests/Qu...",2021,English,ISSN-1741-5659,"Emerald Publishing Limited. Howard House, Wago...",T,NaN
664,EJ1286635,Innovative Pedagogical Strategies of Streaming...,"Larson, Benjamin E., Bohler, Jeffrey A., Krish...",Aim/Purpose: Business analytics is a cross-fun...,"Teaching Methods, Data Analysis, Business Admi...","Journal Articles, Reports - Research",2021,English,ISSN-2165-3151,Informing Science Institute. 131 Brookhill Cou...,T,NaN
